In [1]:
%pip install pyautogen python-dotenv

Note: you may need to restart the kernel to use updated packages.


## Initialize AutoGen LLM Configuration

Ceate a .env file in the root directory and add your **OpenAI API key** to it. This key will be loaded and utilized in the subsequent steps. For the correct format, refer to the `.env.sample` file. (You can also paste your key into this sample file and rename it to `.env`.)

In [12]:
import os
from autogen import filter_config

from dotenv import load_dotenv
load_dotenv()

# This is a list of models that we want to use in the LLM. Each model is a dictionary with some properties.
config_list = [
    # Model that outputs text
    {
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"],
        "response_format": {"type": "text"},
        "tags": ["gpt-4o", "text"]
    }
]

llm_config = {
    "timeout": 600,
    "cache_seed": None,  # Disable caching to make sure we get fresh results. Be careful with this in production. Set this to 41 for disk caching.
    "config_list": config_list,
    "temperature": 1,
}

## Programmar and Peer Review


In [142]:
from autogen.agentchat import AssistantAgent
import pprint

coding_task = "Write a JSON formatting function that removse newlines outside of quotes, and handle JSON escape sequences."

programmar = AssistantAgent(
    name="Programmar",
    is_termination_msg=lambda x: x.get("content", "").find("LGTM") >= 0,
    llm_config=llm_config,
    system_message="""
You are a software engineer who writes code in Typescript. 
You are given a task. You will write code to complete the task.
Keep your messages short.
""",
)

In [143]:
peer_reviewer = AssistantAgent(
    name="PeerReviewer",
    is_termination_msg=lambda x: x.get("content", "").find("LGTM") >= 0,
    llm_config=llm_config,
    system_message="""
You are an expert software engineer who reviews code and provide comments on code quality.
When you are done reviewing the code, write "LGTM" (exact phrase) to indicate that the code is good to merge.
Keep your messages short.
""",
)

In [144]:
res = peer_reviewer.initiate_chat(
    recipient=programmar,
    message=coding_task,
    max_turns=10,
)

PeerReviewer (to Programmar):

Write a JSON formatting function that removse newlines outside of quotes, and handle JSON escape sequences.

--------------------------------------------------------------------------------
Programmar (to PeerReviewer):

Sure, here is a function in TypeScript that removes newlines outside of quotes and handles JSON escape sequences:

```typescript
function removeNewlinesFromJson(jsonString: string): string {
    let result = '';
    let inQuotes = false;

    for (let i = 0; i < jsonString.length; i++) {
        let char = jsonString[i];

        if (char === '"' && (i === 0 || jsonString[i - 1] !== '\\')) {
            inQuotes = !inQuotes;
        }

        if (char === '\n' && !inQuotes) {
            // Skip newlines outside quotes
            continue;
        }

        result += char;
    }

    return result;
}

// Example usage
const jsonString = `
{
    "name": "John",
    "bio": "Loves coding.\nEnjoys coffee."
}
`;
console.log(removeNewlinesFr